In [1]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import logging
import copy

# from importlib import reload # to reload modules
# import modules.LoadBatch
# reload(modules.LoadBatch)
from modules.LoadBatch import *
from modules.SensorClasses import Batch, Sensor, Oscilloscope
from modules.SingleBatch import analysis_batch

In [2]:
logging.basicConfig(level=logging.INFO, format='[%(levelname)s] - %(message)s')
pd.set_option("display.max_columns", None)

### I change every 'nan' that is supposed to be 'Scope X Channel Y' to the correct name (i.e. 'ScopeX ChannelY')

In [3]:
# no clue but it worked
def change_nans(my_header):
    new_columns = my_header
    skip_this = False
    apply_replacement = False

    for idx,name in enumerate(my_header):

        try: 'Scope' in name
        except: pass
        else:
            if 'Scope' in name and 'Channel' in name:
                temp_name = name
                apply_replacement = True
                skip_this = True
            else: apply_replacement = False

        if pd.isna(name) and apply_replacement: skip_this = False
        else: skip_this = True

        if skip_this: continue 

        if apply_replacement: new_columns[idx] = temp_name

    return new_columns

In [18]:
### Read the csv pf the RunLog
# csv_dir = "/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/testbeam-analysis"
csv_dir = "./files/"
csv_name = "CERN Testbeam May 2023 - RunLog.csv"

df = pd.read_csv(os.path.join(csv_dir, csv_name), skiprows=2, header=None, nrows=354)
### Read the CSV file, but only the first two rows (column names)
df_header = pd.read_csv(os.path.join(csv_dir, csv_name), nrows=2, header=None)
df_header = df_header.transpose()
df_header[0] = change_nans(df_header[0])
### Fill the rest of 'nan'
df_header = df_header.fillna('')

### Create a MultiIndex for the columns using the first two rows
multi_index = pd.MultiIndex.from_frame(df_header, names=[None, None])
df.columns = multi_index

### check info
display(df.head())

logging.debug(f"in logbookreader, columns read from the csv: {df.columns}")

### Join the two layers of names with underscores and assign them to the dataframe
new_column_names = [f'{level1}_{level2}' for level1, level2 in df.columns]
df.columns = new_column_names

### clear up headers and col_types
df.columns = df.columns.str.replace(' ', '')
new_column_names = [name.lstrip('_') if name.startswith('_') else name for name in df.columns]
df.columns = new_column_names

df['EUDAQ_RunNumber'] = df['EUDAQ_RunNumber'].fillna(0).astype(int)

print("Batches before removing Ugly or Bad: \n", list(df["Batch"].unique()))

### remove Bad or Ugly Runs
logging.info(f"""length of df: {len(df)}
{df['GoodRun?_G/B/U'].unique()}""")

# df = df[df["GoodRun?_G/B/U"]!='B']  
df = df[df["GoodRun?_G/B/U"]=='G']

logging.info(f"length of df: {len(df)}")
             
dict_batches_path = "./files/dict_of_batches.pickle"


Shifters             EUDAQ       Scope 1  \
          Date  NO COMMAS ANYWHERE PLEASE :)   Time RunNumber     Timestamp   
0  May/26/2023  Khuram - Marcello Pozzessere   9:33      9577  1.685177e+09   
1  May/26/2023  Khuram - Marcello Pozzessere  10:22      9578  1.685179e+09   
2  May/26/2023  Khuram - Marcello Pozzessere  11:34      9579  1.685183e+09   
3  May/26/2023  Khuram - Marcello Pozzessere  12:41      9580  1.685184e+09   
4  May/26/2023  Khuram - Marcello Pozzessere  12:01      9581  1.685184e+09   

        Scope 2                                           #Events         \
      Timestamp Batch This run (triggers field-not event builder)  Total   
0  1.685177e+09   199                                    267839.0  #REF!   
1  1.685179e+09   199                                    327359.0    NaN   
2  1.685183e+09   199                                    302559.0    NaN   
3  1.685184e+09   199                                    178559.0    NaN   
4  1.685184e+09   199                                    312479.0    NaN   

        Av.   SPS values                                      T: DUT1 (left)  \
  Rate [Hz] Intensity T4 Colimators Count on triggers Trigger   T [C] temp A   
0     323.0          NaN        NaN               NaN     NaN          -31.0   
1     347.0          NaN        NaN               NaN     NaN          -31.0   
2     340.0          NaN        NaN               NaN     NaN          -31.0   
3     269.0          NaN        NaN               NaN     NaN          -31.1   
4     337.0          NaN        NaN               NaN     NaN          -31.0   

  T: DUT2 (right)   Humidity: DUT2 (right) Disk space available above 5%?  \
     T [C] temp B Humidity [%] green-right                            Y/N   
0           -31.5                      0.7                            NaN   
1           -31.5                      0.7                            NaN   
2           -31.4                      0.6                            NaN   
3           -31.5                      1.4                            NaN   
4           -31.0                      1.5                            NaN   

          FE-i4 Good Run?          Angle Scope 1 Channel 1                     \
           Mask     G/B/U Angle [degree]          DUT Pos. Sensor  V[V] I [A]   
0  10-70;40-300         G            NaN               5.0    MCP  2600  72.7   
1  10-70;40-300         G            NaN               5.0    MCP  2500  69.1   
2  10-70;40-300         G            NaN               5.0    MCP  2500  69.0   
3  10-70;40-300         G            NaN               5.0    MCP  2500  69.1   
4  10-70;40-300         G            NaN               5.0    MCP  2500  69.1   

  Scope 1 Channel 2                                              \
           DUT Pos.            Sensor V[V] V_real [V]     I [A]   
0               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
1               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
2               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
3               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
4               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   

  Scope 1 Channel 3                                              \
           DUT Pos.            Sensor V[V] V_real [V]     I [A]   
0               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
1               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
2               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
3               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   
4               1.0  IMEv3-W12-C2-2-2  -80      -80.0  0.000004   

  Scope 1 Channel 4                                                  \
           DUT Pos.            Sensor V[V] V_real [V]      I[A]       
0                 1  IMEv3-W12-C3-1-4  -80      -80.0  0.000004 NaN   
1                 1  IMEv3-W12-C3-1-4  -80      -80.0  0.000004 NaN   
2                 1  IMEv3-W12-C3-1-4  -80      -80.0  

[INFO] - length of df: 354
['G' 'B' nan 'U']
[INFO] - length of df: 295


Batches before removing Ugly or Bad: 
 [199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 404, 405, 204, 406, 407, 408, 409, 410, 411, 412, 205, 206, 413, 414, 207, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202, 1203, 1301, 1302, 1303, 1304, 1305, 1306]


In [5]:
print(list(df["Batch"].unique()))

[199, 100, 101, 201, 202, 203, 301, 401, 402, 403, 204, 406, 407, 408, 409, 410, 411, 412, 205, 206, 413, 414, 501, 502, 503, 504, 505, 601, 602, 603, 604, 605, 701, 702, 801, 802, 901, 902, 1001, 1002, 1101, 1102, 1201, 1202, 1301, 1302, 1303, 1304, 1305, 1306]


#### I do not remove the excluded runs yet because they are not the same for both Oscilloscopes
Also I need to remove the runs mostly when I 'hadd' all the tree files together

In [6]:
excluded_runs = {}
                                   # runs 9534 and 9535 have been mislabeled (should have been 9634 and 9635, but still missing)
excluded_runs["S1"] = [9534,9535,9634,9635,
                       9582, 9716, # runs 9582,9716 are missing from S1 and S2 (Ntuples)
                                   # These runs are completely empty (idk why??) 
                       9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739,
                                   # these runs have negative charges, something went wrong (only ch2 and ch3 but have to remove it completely)
                       9747, 9765, 9766, 9767, 9768, 9779, 9780, 9785, 9786,
                       9828,       # run 9828 is missing from S1 (Ntuples)
                       9910,       # run 9910 is empty in S1 and S2 
                       9962, 9963, # weird runs, oscilloscopes might be clipping the pulseHeight
                       10071,      # run 10071 is missing from S1 (Ntuples)

                                   # runs after 10097 are missing from both
                       10110, 10112, 10113, 10114, 10115, 10117,10118,10119,10120,10121,10122,10123,10124]

                                   # runs 9534 and 9535 have been mislabeled (should have been 9634 and 9635, but still missing)
excluded_runs["S2"] = [9534,9535,9634,9635,
                       9582, 9716, # runs 9582,9716 are missing from S1 and S2 (Ntuples)
                                   # These runs are completely empty (idk why??) 
                       9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739,
                       9747, 9765,  # these runs have negative charges, something went wrong

                       9785,       # run 9785 is missing from S2 (Ntuples)
                       9910,       # run 9910 is empty in S1 and S2 
                       9936,       # run 9936 is missing from S2 (Ntuples)
                       10057,      # run 10057 is missing from S2 (Ntuple

                                   # runs after 10097 are missing from both
                       10093, 10093, 10094, 10095, 10096,10110,10112,10113,10114,10115,10117,10118,10119,10120,10121,10122,10123,10124]


## Create the dictionary with improved batches dict

In [7]:
### starting info
df.head()

Date Shifters_NOCOMMASANYWHEREPLEASE:)   Time  EUDAQ_RunNumber  \
0  May/26/2023      Khuram - Marcello Pozzessere   9:33             9577   
1  May/26/2023      Khuram - Marcello Pozzessere  10:22             9578   
2  May/26/2023      Khuram - Marcello Pozzessere  11:34             9579   
3  May/26/2023      Khuram - Marcello Pozzessere  12:41             9580   
4  May/26/2023      Khuram - Marcello Pozzessere  12:01             9581   

   Scope1_Timestamp  Scope2_Timestamp  Batch  \
0      1.685177e+09      1.685177e+09    199   
1      1.685179e+09      1.685179e+09    199   
2      1.685183e+09      1.685183e+09    199   
3      1.685184e+09      1.685184e+09    199   
4      1.685184e+09      1.685184e+09    199   

   #Events_Thisrun(triggersfield-noteventbuilder)  Total  Av._Rate[Hz]  \
0                                        267839.0  #REF!         323.0   
1                                        327359.0    NaN         347.0   
2                                        302559.0    NaN         340.0   
3                                        178559.0    NaN         269.0   
4                                        312479.0    NaN         337.0   

   SPSvalues_IntensityT4  Colimators  Countontriggers  Trigger  \
0                    NaN         NaN              NaN      NaN   
1                    NaN         NaN              NaN      NaN   
2                    NaN         NaN              NaN      NaN   
3                    NaN         NaN              NaN      NaN   
4                    NaN         NaN              NaN      NaN   

   T:DUT1(left)_T[C]tempA  T:DUT2(right)_T[C]tempB  \
0                   -31.0                    -31.5   
1                   -31.0                    -31.5   
2                   -31.0                    -31.4   
3                   -31.1                    -31.5   
4                   -31.0                    -31.0   

   Humidity:DUT2(right)_Humidity[%]green-right  \
0                                          0.7   
1                                          0.7   
2                                          0.6   
3                                          1.4   
4                                          1.5   

   Diskspaceavailableabove5%?_Y/N    FE-i4_Mask GoodRun?_G/B/U  \
0                             NaN  10-70;40-300              G   
1                             NaN  10-70;40-300              G   
2                             NaN  10-70;40-300              G   
3                             NaN  10-70;40-300              G   
4                             NaN  10-70;40-300              G   

   Angle_Angle[degree]  Scope1Channel1_DUTPos. Scope1Channel1_Sensor  \
0                  NaN                     5.0                   MCP   
1                  NaN                     5.0                   MCP   
2                  NaN                     5.0                   MCP   
3                  NaN                     5.0                   MCP   
4                  NaN                     5.0                   MCP   

   Scope1Channel1_V[V]  Scope1Channel1_I[A]  Scope1Channel2_DUTPos.  \
0                 2600                 72.7                     1.0   
1                 2500                 69.1                     1.0   
2                 2500                 69.0                     1.0   
3                 2500                 69.1                     1.0   
4                 2500                 69.1                     1.0   

  Scope1Channel2_Sensor  Scope1Channel2_V[V]  Scope1Channel2_V_real[V]  \
0      IMEv3-W12-C2-2-2                  -80                     -80.0   
1      IMEv3-W12-C2-2-2                  -80                     -80.0   
2      IMEv3-W12-C2-2-2                  -80                     -80.0   
3      IMEv3-W12-C2-2-2                  -80                     -80.0   
4      IMEv3-W12-C2-2-2                  -80                     -80.0   

   Scope1Channel2_I[A]  Scope1Channel3_DUTPos. Scope1Channel3_Sensor  \
0             0.000004

In [8]:
### I put list of runs into Scope so I can have two different run list for the two scopes

### Using the Batch, Oscilloscope and Sensor classes
oscilloscopes = ["Scope1", "Scope2"]
channels = ["Channel1", "Channel2", "Channel3", "Channel4"]
dict_of_batches = {}

for batch in df["Batch"].unique():    # batch number
    temperature_average = []
#     humidity_list = []
    idx = np.argwhere(df["Batch"]==batch)[:,0]  ### take all indices
    logging.info(f"while creating dict_of_batches, Batch:{batch}")
    
    S = {}
    for scope in oscilloscopes:   # scopes
        logging.debug(f'Oscilloscope:{scope}')
        this_scope = scope.replace('Scope','S')
        runs_list_idx = []
        for i in idx:     # runs
            single_run = df["EUDAQ_RunNumber"].iloc[i]
            if single_run in excluded_runs[this_scope]:   continue
            else:   runs_list_idx.append(i)
        runs_list = df["EUDAQ_RunNumber"].iloc[runs_list_idx].to_list()
        logging.debug(f"List of Runs of batch {batch}: {runs_list}")
        for column_name in df.columns:
            if 'tempA' in column_name:    # tempA
                temperatureA = np.array(df[column_name].iloc[runs_list_idx])
            if 'tempB' in column_name:    # tempB
                temperatureB = np.array(df[column_name].iloc[runs_list_idx])
        temperature_average.append(np.hstack((temperatureA,temperatureB)).mean())

        if not runs_list:  ### if it's empty (i.e. all the runs are excluded)
            S[this_scope] = None
            continue 
        
        S[this_scope] = Oscilloscope(this_scope, runs_list, temperatureA, temperatureB) # Oscilloscope object

        for j,ch in enumerate(channels):
            ### I loop over columns names so I can look for strings inside the column name ('scope in column_name')
            for column_name in df.columns:
                logging.debug(f"{column_name}")
                if (scope in column_name) and (ch in column_name):
                    if 'Sensor' in column_name:            # name
                        name = df[column_name].fillna('').iloc[idx[0]]  ### just take the first one
                    elif 'DUTPos' in column_name:           # dut position
                        dut_position = df[column_name].iloc[idx[0]]
                    elif 'V[V]' in column_name:             # voltage
                        voltage = df[column_name].iloc[idx[0]]
                    elif ('I [A]' in column_name) or ('I[A]' in column_name):
                        current = np.abs(df[column_name].iloc[idx[0]])
            this_sensor = Sensor(name, dut_position, voltage, current)
            
            logging.debug(f"while creating dict_of_batches: \n name:{name}, board:{this_sensor.board}, dut pos:{dut_position}, fluence:{this_sensor.fluence}, voltage:{voltage}")
            S[this_scope].add_sensor(f"Ch{j+1}",this_sensor)
    for column_name in df.columns:
        if 'Angle' in column_name:    # angle
            angle = np.average(df[column_name].iloc[runs_list_idx].dropna())
        if 'Humidity' in column_name:
            humidity = np.average(df[column_name].iloc[runs_list_idx].dropna())
    temperature = np.average(temperature_average)
    
    dict_of_batches[batch] = Batch(batch,angle,humidity,temperature,S['S1'],S['S2'])
#     dict_of_batches[batch] = Batch(batch,angle,temperature_average,temperatureA,temperatureB,S['S1'],S['S2'])
    
    
### I need to merge batches 403 and 406 (as from notes) (NO)
### I CANNOT DO IT LIKE THIS, THEY LOOK LIKE SHIT HOW DID I NOT NOTICE BEFORE
### (WHICH ONES???? 403 OR 406??) they only looked bad once merged

### I need to merge batches 405 and 407 (as from notes)
### nevermind batch 405 is all ugly

### manually separate 503 into 503.1 and 503.2 (now it's float but should work anyways, hopefully)
### it did not, I change it now to 5031 and 5032
dict_of_batches[5031] = copy.deepcopy(dict_of_batches[503])
dict_of_batches[5031].batch_number = 5031
dict_of_batches[5031].S['S1'].runs = [9829, 9833, 9834, 9835]
dict_of_batches[5031].S['S2'].runs = [9828, 9829, 9833, 9834, 9835]
dict_of_batches[5031].S['S1'].tempA = [-26.2, -25.6, -25.4, -25.3]
dict_of_batches[5031].S['S1'].tempB = [-26.4, -25.8, -25.6, -25.5]
dict_of_batches[5031].S['S2'].tempA = [-26.3, -26.2, -25.6, -25.4, -25.3]
dict_of_batches[5031].S['S2'].tempB = [-26.5, -26.4, -25.8, -25.6, -25.5]


dict_of_batches[5032] = copy.deepcopy(dict_of_batches[503])
dict_of_batches[5032].batch_number = 5032
dict_of_batches[5032].S['S1'].runs = [9843, 9844, 9845, 9846, 9847]
dict_of_batches[5032].S['S2'].runs = [9843, 9844, 9845, 9846, 9847]
dict_of_batches[5032].S['S1'].tempA = [-33.1, -33.2, -33.2, -33.2, -33.2]
dict_of_batches[5032].S['S1'].tempB = [-33.4, -33.4, -33.4, -33.5, -33.5]
dict_of_batches[5032].S['S2'].tempA = [-33.1, -33.2, -33.2, -33.2, -33.2]
dict_of_batches[5032].S['S2'].tempB = [-33.4, -33.4, -33.4, -33.5, -33.5]
del dict_of_batches[503]
    
# ### Save the sensor list into a file
### I should not save here, only after all additional changes are applied
# dict_batches_path = "./files/dict_of_batches.pickle"
# with open(dict_batches_path, 'wb') as f:
#     pickle.dump(dict_of_batches, f)

# logging.info(f"{dict_of_batches.keys()}")


[INFO] - while creating dict_of_batches, Batch:199
/home/marcello/.local/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/home/marcello/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:100
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:101
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batc

[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:505
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:601
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:602
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:603
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance 

[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:1304
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - while creating dict_of_batches, Batch:1305
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_transimpedance(): No board name assigned: no transimpedance set
[INFO] - in set_

Remove empty batches (i.e. the batches that ha all of the runs in 'excluded runs')


In [9]:
batches_to_remove = []
### find all the batches that are empty
for key, batch_object in dict_of_batches.items():
    if (batch_object.S['S1'] is None) and (batch_object.S['S2'] is None):
        batches_to_remove.append(key)
### remove them
for key in batches_to_remove:
    del dict_of_batches[key]

In [10]:
### print which batch the missing runs belong to
### now the missing runs are in each oscilloscope

# for key, value in dict_of_batches.items():
#     for single_run in value.runs:
#         if (single_run in excluded_runs['S1']) or (single_run in excluded_runs['S2']):
#             print(f"run {single_run} missing from batch {key}, only one scope")

In [11]:
### check that the batches 403 and 406 that I merged have the same values 
# for S in ('S1','S2'):
#     print(S)
#     print(dict_of_batches[403].__dict__)
#     print(dict_of_batches[406].__dict__)
#     for i in range(1,5):
#         print(dict_of_batches[403].S[S].get_sensor(f'Ch{i}').__dict__)
#         print(dict_of_batches[406].S[S].get_sensor(f'Ch{i}').__dict__)
        
###         (and 503.1 and 503)
# for S in ('S1','S2'):
#     print(S)
#     print(dict_of_batches[503].__dict__)
#     print(dict_of_batches[503.1].__dict__)
#     for i in range(1,5):
#         print(dict_of_batches[503].S[S].get_sensor(f'Ch{i}').__dict__)
#         print(dict_of_batches[503.1].S[S].get_sensor(f'Ch{i}').__dict__) 


### I need to manually fix some of the sensor names:
I already fixed some names manually from the .csv file, but now I change it here to keep track


In [12]:
### these were both named 'CERN2-CH0-IMEv3-W12', but it's two channels
### furthermore the previous batch was started 10 minutes before so they could not have changed positions
print('Ch2:',dict_of_batches[1002].S['S1'].get_sensor('ch2').name)
print('Ch3:',dict_of_batches[1002].S['S1'].get_sensor('ch3').name)
print('Ch4:',dict_of_batches[1002].S['S1'].get_sensor('ch4').name)

dict_of_batches[1002].S['S1'].get_sensor('ch3').name = "CERN2-CH1-IMEv3-W12"
dict_of_batches[1002].S['S1'].get_sensor('ch4').name = "CERN2-CH2-IMEv3-W12"


Ch2: CERN2-CH0-IMEv3-W12
Ch3: CERN2-CH0-IMEv3-W12
Ch4: CERN2-CH0-IMEv3-W12


In [13]:
### in a few batches (5xx and 12xx, and only on channel3) the name is only JSI-PP1
### but it's a CNM sensor with 1.5E15 fluence on W5 so I rename it accordingly
### OF COURSE IT SHOULD DO NOTHING IF IT HAS ALREADY RUN ONCE

new_name = 'JSI-PP1-CNM-W5-1.5E15'
for this_batch, batch_object in dict_of_batches.items():
    for S in ('S1','S2'):
        if dict_of_batches[this_batch].S[S].get_sensor('ch3').name == 'JSI-PP1':
            logging.warning(f"in logbookreader, renaming Ch3 in batch: {this_batch} from JSI-PP1' to: {new_name}")
            dict_of_batches[this_batch].S[S].get_sensor('ch3').name = new_name


[WARNING] - in logbookreader, renaming Ch3 in batch: 501 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 502 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 504 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 505 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 1201 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 1202 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 5031 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15
[WARNING] - in logbookreader, renaming Ch3 in batch: 5032 from JSI-PP1' to: JSI-PP1-CNM-W5-1.5E15


In [14]:
### I think I want to rename them to a more understandable naming convention
unique_names = []
for batch_object in dict_of_batches.values():
    for S in ['S1','S2']:
        if batch_object.S[S] is not None:
            for ch, sensor in batch_object.S[S].channels.items():
                print(sensor.name)
                if (sensor.name != '') and (sensor.name not in unique_names):
                    unique_names.append(sensor.name)
logging.info(f"{unique_names}")


[INFO] - ['MCP', 'IMEv3-W12-C2-2-2', 'IMEv3-W12-C3-1-4', 'CNM-R15973-W5-D138', 'CNM-R15973-W4-D168', 'IMEv3-W12-C3-1-5', 'USTC2.1-W17-P6-A', 'no_board', 'JSI-PP4-IMEv2-W7-6.5E14', 'JSI-B7-IMEv3-W16-8E14', 'JSI-B13-IMEv3-W16-2.5E15', 'JSI-B6-IMEv2-W7-1E14', 'JSI-PP1-CNM-W5-1.5E15', 'CERN2-CH0-IMEv3-W12', 'CERN2-CH1-IMEv3-W12', 'CERN2-CH2-IMEv3-W12', 'JSI-B5-CNM-W3-2.5E15', 'CERN2-CH4-IMEv3-W16']


MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
IMEv3-W12-C2-2-2
IMEv3-W12-C2-2-2
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
USTC2.1-W17-P6-A
USTC2.1-W17-P6-A
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
USTC2.1-W17-P6-A
USTC2.1-W17-P6-A
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973-W4-D168
IMEv3-W12-C3-1-5
MCP
USTC2.1-W17-P6-A
USTC2.1-W17-P6-A
IMEv3-W12-C3-1-4
MCP
CNM-R15973-W5-D138
CNM-R15973

In [15]:
### rename the sensors:
renaming_dict = {'MCP':'MCP',
                 'CNM-R15973-W5-D138':'CNM-W5',
                 'CNM-R15973-W4-D168':'CNM-W4',
                 'JSI-PP1-CNM-W5-1.5E15':'CNM-W5-1.5E15',
                 'JSI-B5-CNM-W3-2.5E15':'CNM-W3-2.5E15',
                 'USTC2.1-W17-P6-A':'USTC2.1-W17',
                 'IMEv3-W12-C2-2-2':'IMEv3-W12-2x2',
                 'IMEv3-W12-C3-1-4':'IMEv3-W12-1x3',
                 'IMEv3-W12-C3-1-5':'IMEv3-W12-1x3',
                 'CERN2-CH0-IMEv3-W12':'IMEv3-W12-2x2-1.5E15',
                 'CERN2-CH1-IMEv3-W12':'IMEv3-W12-2x2-1.5E15',
                 'CERN2-CH2-IMEv3-W12':'IMEv3-W12-2x2-1.5E15',
                 'CERN2-CH4-IMEv3-W16':'IMEv3-W16-1x3-1.5E15',
                 'JSI-B6-IMEv2-W7-1E14':'IMEv2-W7-1E14',
                 'JSI-PP4-IMEv2-W7-6.5E14':'IMEv2-W7-6.5E14',
                 'JSI-B7-IMEv3-W16-8E14':'IMEv3-W16-8E14',
                 'JSI-B13-IMEv3-W16-2.5E15':'IMEv3-W16-2.5E15',
                 NO_BOARD:NO_BOARD}

for batch_object in dict_of_batches.values():
    for S in ['S1','S2']:
        if batch_object.S[S] is not None:
            for ch, sensor in batch_object.S[S].channels.items():
                old_name = sensor.name
                new_name = renaming_dict.get(old_name)
                if old_name in renaming_dict.values():
                    logging.info(f"Name already changed: {old_name}")
                else:
                    if new_name is None:
                        logging.warning(f"The new name is not contained in the dictionary, old name: {old_name}")
                    else:
                        sensor.name = new_name
                        print('name after changing:', dict_of_batches[this_batch].S[S].channels[ch].name)
                

[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name alread

name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W

[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[WARNING] - The new name is not contained in the dictionary, old name: 
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[WARNING] - The new name is not contained in the dictionary, old name: 
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[WARNING] - The new name is not contained in the dictionary, old name: 
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[WARNING] - The new name is not contained in the dictionary, old name: 
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Name already changed: MCP
[INFO] - Nam

name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: 
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after changing: JSI-PP1-CNM-W5-1.5E15
name after changing: JSI-PP4-IMEv2-W7-6.5E14
name after changing: JSI-B7-IMEv3-W16-8E14
name after changing: JSI-B13-IMEv3-W16-2.5E15
name after changing: JSI-B6-IMEv2-W7-1E14
name after chang

In [16]:
### I think I want to rename them to a more understandable naming convention
unique_names = []
for batch_object in dict_of_batches.values():
    for S in ['S1','S2']:
        if batch_object.S[S] is not None:
            for ch, sensor in batch_object.S[S].channels.items():
                if (sensor.name != '') and (sensor.name not in unique_names):
                    unique_names.append(sensor.name)
                    print(sensor.name)
logging.info(f"{unique_names}")


[INFO] - ['MCP', 'IMEv3-W12-2x2', 'IMEv3-W12-1x3', 'CNM-W5', 'CNM-W4', 'USTC2.1-W17', 'no_board', 'IMEv2-W7-6.5E14', 'IMEv3-W16-8E14', 'IMEv3-W16-2.5E15', 'IMEv2-W7-1E14', 'CNM-W5-1.5E15', 'IMEv3-W12-2x2-1.5E15', 'CNM-W3-2.5E15', 'IMEv3-W16-1x3-1.5E15']


MCP
IMEv3-W12-2x2
IMEv3-W12-1x3
CNM-W5
CNM-W4
USTC2.1-W17
no_board
IMEv2-W7-6.5E14
IMEv3-W16-8E14
IMEv3-W16-2.5E15
IMEv2-W7-1E14
CNM-W5-1.5E15
IMEv3-W12-2x2-1.5E15
CNM-W3-2.5E15
IMEv3-W16-1x3-1.5E15


## Save the updated file

In [19]:
with open(dict_batches_path, 'wb') as f:
    pickle.dump(dict_of_batches, f)

## Now using dict_of_batches

In [20]:
create_combined_tree_files = False ### so I don't accidentally run it everytime

try:
    os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1")
    !pwd
    for key, value in dict_of_batches.items():
        print("batch:", key)
        output_name = f"tree_May2023_S1_{key}.root"
        merge_command = f"hadd -f {output_name}"
        for run in value.S['S1'].runs:
            if run not in excluded_runs['S1']:
                if run<=9999:       merge_command += f" tree00{run}.root"
                elif run>=10000:    merge_command += f" tree0{run}.root"
        print(merge_command)
        if create_combined_tree_files:    os.system(merge_command)
except: logging.error("something wrong")

/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S1
batch: 199
hadd -f tree_May2023_S1_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
batch: 100
hadd -f tree_May2023_S1_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
batch: 101
hadd -f tree_May2023_S1_101.root tree009574.root
batch: 201
hadd -f tree_May2023_S1_201.root tree009587.root tree009588.root tree009590.root
batch: 202
hadd -f tree_May2023_S1_202.root tree009591.root tree009592.root tree009593.root tree009594.root
batch: 203
hadd -f tree_May2023_S1_203.root tree009595.root tree009596.root tree009602.root
batch: 301
hadd -f tree_May2023_S1_301.root tree009609.root tree009611.root tree009613.root tree009614.root tree009615.root tree009616.root
batch: 401
hadd -f tree_May2023_S1_401.root tree009620.root tree009621.root tree009622.root tree009624.root tree009625.root tree009627.root tree009628.root tree009629.root tree0

In [36]:
try:
    os.chdir("/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2")
    !pwd
    for key, value in dict_of_batches.items():
        print("batch:", key)
        output_name = f"tree_May2023_S2_{key}.root"
        merge_command = f"hadd -f {output_name}"
        for run in value.S['S2'].runs:
            if run not in excluded_runs['S2']:
                if run<=9999:       merge_command += f" tree00{run}.root"
                elif run>=10000:    merge_command += f" tree0{run}.root"
        print(merge_command)
        if create_combined_tree_files:    os.system(merge_command)
except: logging.error("something wrong")

/home/marcello/Desktop/Radboud_not_synchro/Master_Thesis/Data_TestBeam/2023_May/S2


[ERROR] - something wrong


batch: 199
hadd -f tree_May2023_S2_199.root tree009577.root tree009578.root tree009579.root tree009580.root tree009581.root
batch: 100
hadd -f tree_May2023_S2_100.root tree009564.root tree009565.root tree009567.root tree009568.root tree009570.root
batch: 101
hadd -f tree_May2023_S2_101.root tree009574.root
batch: 201
hadd -f tree_May2023_S2_201.root tree009587.root tree009588.root tree009590.root
batch: 202
hadd -f tree_May2023_S2_202.root tree009591.root tree009592.root tree009593.root tree009594.root
batch: 203
hadd -f tree_May2023_S2_203.root tree009595.root tree009596.root tree009602.root
batch: 301
hadd -f tree_May2023_S2_301.root tree009609.root tree009611.root tree009613.root tree009614.root tree009615.root tree009616.root
batch: 401
hadd -f tree_May2023_S2_401.root tree009620.root tree009621.root tree009622.root tree009624.root tree009625.root tree009627.root tree009628.root tree009629.root tree009630.root
batch: 402
hadd -f tree_May2023_S2_402.root tree009631.root tree009632.r